# GDML Explorer

Author: [Sam Eriksen](mailto:sam.eriksen@bristol.ac.uk)

## Purpose
To work out how to explore GDML files to convert to trees for GPU simulations.

##### Unusual Libraries Needed
* lxml

### GDML Playing

Start by understanding the structure and using the GDML structure.
Eventually I'll turn this into a class (similar to opticks approach)

A good place to start is the [lxml tutorial](https://lxml.de/tutorial.html)

In [13]:
from lxml import etree
import numpy as np

In [232]:
#
gdml_file = "/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml"
gdml_e = etree.parse(gdml_file).getroot()
print('gdml element:',gdml_e)

# Get info of element
print('\nGDML structure\n--------------')
for child in gdml_e:
    print(child.tag)

gdml element: <Element gdml at 0x2aaacd86ac48>

GDML structure
--------------
define
materials
solids
structure
userinfo
setup


## Learning Navigation 
Write a class to extract different information

In [233]:
from lxml import etree
class GDML():
    
    def __init__(self, path):
        
        self.path = path
        self.fulletree = etree.parse(self.path).getroot()
        self.materials = self._findallnames("materials/material")
        self.nmaterials = len(self.materials)
        self.structures = self._findallnames("structure/volume")
        self.nstructures = len(self.structures)
        self.world = self._findWorld()
        
    def _findallnames(self, tofind):
        allthings = self.fulletree.findall(tofind)
        valuelist = []
        for value in allthings:
            attribute = value.attrib
            valuelist.append(attribute['name'])
        return valuelist
        
    def _findWorld(self):
        world_setup = self.fulletree.findall("setup/world")
        world = world_setup[0].attrib['ref']
        return(world)


In [234]:
a = GDML("/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml")

In [235]:
a.path

'/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml'

In [236]:
a.nstructures

8828

# Handling the Entire File 

Before turning into a binary tree, have to extract more information out and link it.

Firstly, create a class similar to the one above which handles the elements from the etree in a better way.
These elements can be a number of different things including, eg a Material or a Volume

In [288]:
class GDMLElement(object):

    def __init__(self, elem, g=None):
        self.elem = elem
        self.g = g
        self.name = self.elem.attrib.get('name',None)
        self.typ = self.__class__.__name__
        self.gidx = (lambda self:"[%d]" % self.idx if hasattr(self, 'idx') else '[]' )
        self.idx = []
        self.name = lambda self:self.elem.attrib.get('name',None)
        
    def _findall(self, expr):
        wrap_ = lambda e: print(self.g.kls.get(e.tag,GDMLElement)) #(e,self.g)
        fa = map(wrap_, self.elem.findall(expr) )
        return fa
    
    def att(self, name, default=None, typ=None):
        assert typ is not None
        v = self.elem.attrib.get(name, default)
        return typ(v)

Next need to handle each of the types of things which will be in the GDML file.
A class is needed for each of these

In [289]:
import collections
class odict(collections.OrderedDict):
    def __call__(self, iarg):
        return self.items()[iarg][1]

Collect Materials

In [290]:
class Material(GDMLElement):
    
    def __init__(self):
        self.state = self.att('state', '', type=str)
        
    def __repr__(self):
        return "%s %s %s %s" % (self.gidx, self.typ, self.name, self.state )

In [291]:
class Volume(GDMLElement):

    def __init__(self):
        self.materialref = lambda self:self.elem.find("materialref").attrib["ref"]
        self.solidref = lambda self:self.elem.find("solidref").attrib["ref"]
        self.solid = (lambda self:self.g.solids[self.solidref])
        self.material = (lambda self:self.g.materials[self.materialref])
        self.physvol = (lambda self:self.findall_("physvol"))
        self.children = (lambda self:self.physvol)

    def filterpv(self, pfx):
        return filter(lambda pv:pv.name.startswith(pfx), self.physvol) 

    def rdump(self, depth=0):
        for pv in self.physvol:
            lv = pv.volume
            lv.rdump(depth=depth+1)
 

    def __repr__(self):
        repr_ = lambda _:"   %r" % _ 
        pvs = map(repr_, self.physvol) 
        line = "%s %s %s %s %s" % (self.gidx, self.typ, self.name, self.materialref, self.solidref)
        return "\n".join([line, repr_(self.solid), repr_(self.material)] + pvs )
    
class PhysVol(GDMLElement):

    def __init__(self):
        self.volumeref = (lambda self:self.elem.find("volumeref").attrib["ref"])
        self.volume = (lambda self:self.g.volumes[self.volumeref])
        self.children = (lambda self:[self.volume])

        self.position = (lambda self:self.find1_("position"))
        self.rotation = (lambda self:self.find1_("rotation"))
        self.scale = (lambda self:self.find1_("scale"))
        self.transform = (lambda self:construct_transform(self))

    def __repr__(self):
        return "\n".join(["%s %s" % (self.typ, self.name)," %r %r " % ( self.position, self.rotation)])

In [292]:
parse_ = lambda _:etree.parse(os.path.expandvars(_)).getroot()
tostring_ = lambda _:etree.tostring(_)
import os
class GDML(GDMLElement):
# Types of things in the
    kls = {
            "material":Material,
            "volume":Volume,
            "physvol":PhysVol,
        }
    @classmethod
    def parse(cls, path="/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml"):
        gg = parse_(path)
        a = GDMLElement(gg)
        wgg = cls.wrap(gg, path=path)
        return wgg 

    @classmethod
    def fromstring(cls, st ):
        gg = ET.fromstring(st) 
        wgg = cls.wrap(gg)
        return wgg 

    @classmethod
    def wrap(cls, gdml, path=None):
        gg = cls(gdml)
        gg.g = gg
        gg.path = path
        gg.string = tostring_(gdml)
        gg.init()
        return gg 
    
    def find_by_prefix(self, d, prefix):
        return filter(lambda v:v.name.startswith(prefix), d.values())
    
    def find_materials(self, prefix="/dd/Materials/Acrylic"):
        return self.find_by_prefix(self.materials, prefix)
    
    def find_volumes(self, prefix="/dd/Geometry/PMT/lvPmtHemi"):
        return self.find_by_prefix(self.volumes, prefix)
    
    world = property(lambda self:self.volumes[self.worldvol])

    def init(self):
    
        self.materials = odict()
        self.volumes = odict()
        
        for i, e in enumerate(self._findall("materials/material")):
            e.idx = i
            self.materials[e.name] = e 
        pass
        for i, e in enumerate(self._findall("structure/*")):
            e.idx = i
            self.volumes[e.name] = e
        pass

In [293]:
a = GDML.parse()

<class '__main__.Material'>


AttributeError: 'NoneType' object has no attribute 'idx'

In [259]:
a.find_materials("tyvek6000xea1f50")

AttributeError: 'GDML' object has no attribute 'find_materials'

In [213]:
a.volumes

odict()

In [54]:
a.find_materials("quartz0xe969e0")

In [57]:
a.materials[a.find_materials("quartz0xe969e0")]

KeyError: <filter object at 0x2aaacd2de278>